# Parsing PDFs Homework

With the power of pdfminer, pytesseract, Camelot, and Tika, let's analyze some documents!

> If at any point you think, **"I'm close enough, I'd just edit the rest of it in Excel"**: that's fine! Just make a note of it.

## A trick to use again and again

### Approach 1

Before we get started: when you want to take the first row of your data and set it as the header, use this trick.

In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame([
    [ 'fruit name', 'likes' ],
    [ 'apple', 15 ],
    [ 'carrot', 3 ],
    [ 'sweet potato', 45 ],
    [ 'peach', 12 ],
])
df

,0,1
0,fruit name,likes
1,apple,15
2,carrot,3
3,sweet potato,45
4,peach,12


In [3]:
# Set the first column as the columns
df.columns = df.loc[0]

# Drop the first row
df = df.drop(0)

df

,fruit name,likes
1,apple,15
2,carrot,3
3,sweet potato,45
4,peach,12


🚀 Done!

### Approach 2

Another alternative is to use `.rename` on your columns and just filter out the columns you aren't interested in. This can be useful if the column name shows up multiple times in your data for some reason or another.

In [4]:
# Starting with the same-ish data...
df = pd.DataFrame([
    [ 'fruit name', 'likes' ],
    [ 'apple', 15 ],
    [ 'carrot', 3 ],
    [ 'fruit name', 'likes' ],
    [ 'sweet potato', 45 ],
    [ 'peach', 12 ],
])
df

,0,1
0,fruit name,likes
1,apple,15
2,carrot,3
3,fruit name,likes
4,sweet potato,45
5,peach,12


In [5]:
df = df.rename(columns={
    0: 'fruit name',
    1: 'likes'
})
df = df[df['fruit name'] != 'fruit name']
df

,fruit name,likes
1,apple,15
2,carrot,3
4,sweet potato,45
5,peach,12


🚀 Done!

### Useful tips about coordinates

If you want to grab only a section of the page [Kull](https://jsoma.github.io/kull/#/) might be helpful in finding the coordinates.

> **Alternatively** run `%matplotlib notebook` in a cell. Afterwards, every time you use something like `camelot.plot(tables[0]).show()` it will get you nice zoomable, hoverable versions that include `x` and `y` coordinates as you move your mouse.

Coordinates are given as `"left_x,top_y,right_x,bottom_y"` with `(0,0)` being in the bottom left-hand corner.

Note that all coordinates are strings, for some reason. It won't be `[1, 2, 3, 4]` it will be `['1,2,3,4']`

# Camelot questions

The largest part of this assignment is **mostly Camelot work**. As tabular data is usually the most typical data you'll be working with, it's what I'm giving you!

It will probably be helpful to read through [Camelot's advanced usage tips](https://camelot-py.readthedocs.io/en/master/user/advanced.html), along with the notebook I posted in the homework assignment.

## Prison Inmates

Working from [InmateList.pdf](InmateList.pdf), save a CSV file that includes every inmate.

* Make sure your rows are *all data*, and you don't have any people named "Inmate Name."


In [6]:
import requests
from bs4 import BeautifulSoup

/Users/onlyandrewn/.pyenv/versions/3.10.3/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [7]:
# !pip install "camelot-py[base]"

You should consider upgrading via the '/Users/onlyandrewn/.pyenv/versions/3.10.3/bin/python3.10 -m pip install --upgrade pip' command.


In [8]:
import camelot

In [9]:
# !pip install opencv-python

You should consider upgrading via the '/Users/onlyandrewn/.pyenv/versions/3.10.3/bin/python3.10 -m pip install --upgrade pip' command.


In [268]:
tables = camelot.read_pdf("InmateList.pdf", flavor="stream", pages="all")

for table in tables:    
    if table.df.shape[0] == 47:
        table.df = table.df \
            .drop([0, 1, 2, 46])
        
        if table.df.shape[1] == 6:
            table.df = table.df \
                .drop([2, 5], axis=1) \
                .rename(columns={
                    0: "ICN #",
                    1: "Inmate Name",
                    3: "Facility",
                    4: "Booking Date"
                })
        elif table.df.shape[1] == 5:
            table.df = table.df \
                .drop([4], axis=1) \
                .rename(columns={
                    0: "ICN #",
                    1: "Inmate Name",
                    2: "Facility",
                    3: "Booking Date"
                })

    if table.df.shape[0] == 42:
        table.df = table.df \
            .drop([0, 1, 2]) \
            .rename(columns={
                0: "ICN #",
                1: "Inmate Name",
                2: "Facility",
                3: "Booking Date"
            })

In [270]:
dfs = [table.df for table in tables]

combined_dfs = pd.concat(dfs, ignore_index=True)
combined_dfs.to_csv("combined.csv", index=False)

## WHO resolutions

Using [A74_R13-en.pdf](A74_R13-en.pdf), what ten member countries are given the highest assessments?

* You might need to have two separate queries, and combine the results: that last page is pretty awful!
* Always rename your columns
* Double-check that your sorting looks right......
* You can still get the answer even without perfectly clean data

In [272]:
tables = camelot.read_pdf("A74_R13-en.pdf", flavor="stream", pages="all")

tables[5].df = tables[5].df \
    .drop([0, 1, 2, 3, 6, 7, 8, 9]) \
    .drop([0, 2, 4], axis=1) \
    .rename(columns={
        1: "Members and Associate Members",
        3: "WHO scale for 2022–2023 %"
    })

for table in tables[:5]:
    table.df = table.df \
        .drop([0, 1, 2]) \
        .rename(columns={
            0: "Members and Associate Members",
            1: "WHO scale for 2022–2023 %"
        })

In [273]:
dfs_who = [table.df for table in tables]
combined_dfs = pd.concat(dfs_who, ignore_index=True)
combined_dfs.to_csv("combined_who.csv")

In [281]:
combined_dfs.dtypes

Members and Associate Members    object
WHO scale for 2022–2023 %        object
dtype: object

In [282]:
combined_dfs.sort_values(by="WHO scale for 2022–2023 %", ascending=False).head(10)

,Members and Associate Members,WHO scale for 2022–2023 %
88,Japan,8.5645
68,Germany,6.0904
191,Northern Ireland,4.5673
64,France,4.4273
86,Italy,3.3072
193,United States of America,22.0000
23,Brazil,2.9482
31,Canada,2.7342
148,Russian Federation,2.4052
145,Republic of Korea,2.2671


## The Avengers

Using [THE_AVENGERS.pdf](THE_AVENGERS.pdf), approximately how many lines does Captain America have as compared to Thor and Iron Man?

* Character names only: we're only counting `IRON MAN` as Iron Man, not `TONY`.
* Your new best friend might be `\n`
* Look up `.count` for strings

In [285]:
from pdfminer.high_level import extract_text

In [286]:
text = extract_text("THE_AVENGERS.pdf")

In [287]:
text[:1000]

'Marvel’s THE AVENGERS\n\nWritten By\n\nJoss Whedon\n\nStory By\nZak Penn and Joss Whedon\n\nBased on the characters appearing in the comic books\nPublished by MARVEL Comics\n\nTranscribed to PDF using:\nCeltx Studio | 2012\n\nComposed In Celtx Studio\nWith the help of\nSiOmniaFicta’s Script\non fanfiction.net\n\nENTERTAINMENT PURPOSES ONLY\n\n\x0c1\n\nEXT. UNKNOWN AREA OF SPACE\n\nSpace, a floating staircase among the rocks...THE OTHER\nspeaks to a DISEMBODIED FIGURE in a chair.\n\nTHE OTHER (V.O)\n\nThe Tesseract has awakened. It is\non a little world, a human world.\nThey would wield its power.\n\nTHE OTHER hands LOKI a scepter with a glowing blue gem...\n\nTHE OTHER (V.O)\nBut our ally knows its workings as\nthey never will. He is ready to\nlead. And our force...\n\nThe CHITAURI put on their face mask and get ready.\n\nTHE OTHER (V.O)\n\n...our Chitauri, will follow. The\nworld will be his...the universe,\nyours. And the humans, what can\nthey do but burn?\n\nA 3d model of the Tess

In [307]:
print(text.count("IRON MAN"))
print(text.count("THOR"))
print(text.count("CAPTAIN AMERICA"))

52
49
40


## COVID data

Using [covidweekly2721.pdf](covidweekly2721.pdf), what's the total number of tests performed in Minnesota? Use the Laboratory Test Rates by County of Residence chart.

* You COULD pull both tables separately OR you could pull them both at once and split them in pandas.
* Remember you can do things like `df[['name','age']]` to ask for multiple columns

In [340]:
tables = camelot.read_pdf("covidweekly2721.pdf", flavor="stream", pages="6")

tables[0].df = tables[0].df \
    .drop([0, 1, 2, 3, 4, 6, 7, 12, 52, 53]) \
    .drop(0, axis=1)

table_1 = tables[0].df[[1, 2, 3]] \
    .rename(columns={
        1: "County",
        2: "Number of Tests",
        3: "Cumulative Rate"
    })

table_2 = tables[0].df[[4, 5, 6]] \
    .rename(columns={
        4: "County",
        5: "Number of Tests",
        6: "Cumulative Rate"
    })


table_1.iloc[4, table_1.columns.get_loc('Cumulative Rate')] = "19,574"

merged = pd.concat([table_1, table_2], ignore_index=True)
merged

,County,Number of Tests,Cumulative Rate
0,Aitkin,"19,204","12,128"
1,Anoka,"545,958","15,714"
2,Becker,"59,238","17,540"
3,Beltrami,"60,345","13,085"
4,Benton,"77,865","19,574"
...,...,...,...
83,Wilkin,"7,973","12,570"
84,Winona,"106,625","20,970"
85,Wright,"194,085","14,621"
86,Yellow Medicine,"19,972","20,239"


## Theme Parks

Using [2019-Theme-Index-web-1.pdf](2019-Theme-Index-web-1.pdf), save a CSV of the top 10 theme park groups worldwide.

* You can clean the results or you can restrict the area the table is pulled from, up to you

In [366]:
tables = camelot.read_pdf("2019-Theme-Index-web-1.pdf", flavor="stream", pages="11")

tables[0].df = tables[0].df \
    .drop([10, 11]) \
    .rename(columns={
        0: "Rank",
        1: "Group Name",
        2: "% Change",
        3: "Attendance 2019",
        4: "Attendance 2018"
    })

tables[0].df.loc[0, "Rank": "Attendance 2018"] = ["1", "WALT DISNEY ATTRACTIONS", "0.8%", "155,991,000", "157,311,000"]

,Rank,Group Name,% Change,Attendance 2019,Attendance 2018
0,1,WALT DISNEY ATTRACTIONS,0.8%,"155,991,000","157,311,000"
1,2,MERLIN ENTERTAINMENTS GROUP,0.9%,"67,000,000","66,400,000*"
2,3,OCT PARKS CHINA,9.4%,"53,970,000","49,350,000"
3,4,UNIVERSAL PARKS AND RESORTS,2.3%,"51,243,000","50,068,000"
4,5,FANTAWILD GROUP,19.8%,"50,393,000","42,074,000"
5,6,CHIMELONG GROUP,8.9%,"37,018,000","34,007,000"
6,7,SIX FLAGS INC.,2.5%,"32,811,000","32,024,000"
7,8,CEDAR FAIR ENTERTAINMENT COMPANY,7.8%,"27,938,000","25,912,000"
8,9,SEAWORLD PARKS & ENTERTAINMENT,0.2%,"22,624,000","22,582,000"
9,10,PARQUES REUNIDOS,6.2%,"22,195,000","20,900,000"


## Hunting licenses

Using [US_Fish_and_Wildlife_Service_2021.pdf](US_Fish_and_Wildlife_Service_2021.pdf) and [a CSV of state populations](http://goodcsv.com/geography/us-states-territories/), find the states with the highest per-capita hunting license holders.

In [370]:
tables = camelot.read_pdf("US_Fish_and_Wildlife_Service_2021.pdf")

tables[0].df \
    .drop([0, 57]) \
    .rename(columns={
        0: "State",
        1: "Paid Hunting License Holders",
        2: "Resident Hunting Licenses, Tags, Permits and Stamps",
        3: "Non-Resident Hunting Licenses, Tags, Permits and Stamps",
        4: "Total Hunting License, Tags,Permits & Stamps",
        5: "Cost - Resident Hunting Licenses, Tags, Permits and Stamps",
        6: "Cost - Non-Resident Hunting Licenses, Tags, Permits and Stamps",
        7: "Gross Cost - Hunting Licenses"
    })

,State,Paid Hunting License Holders,"Resident Hunting Licenses, Tags, Permits and Stamps","Non-Resident Hunting Licenses, Tags, Permits and Stamps","Total Hunting License, Tags,Permits & Stamps","Cost - Resident Hunting Licenses, Tags, Permits and Stamps","Cost - Non-Resident Hunting Licenses, Tags, Permits and Stamps",Gross Cost - Hunting Licenses
1,AK,"93,559","423,501","59,235","482,736","$4,859,356","$9,046,715","$13,906,071"
2,AL,"452,400","601,683","45,397","647,080","$9,700,295","$6,715,734","$16,416,029"
3,AR,"343,300","349,098","150,728","499,826","$7,851,601","$11,271,653","$19,123,254"
4,AS,0,0,0,0,$0,$0,$0
5,AZ,"302,383","464,607","88,708","553,315","$13,931,397","$5,968,169","$19,899,566"
6,CA,"262,009","949,540","25,535","975,075","$20,270,941","$1,114,625","$21,385,566"
7,CO,"296,609","481,253","120,954","602,207","$13,270,492","$49,732,815","$63,003,307"
8,CT,"32,052","101,377","4,543","105,920","$1,855,046","$372,108","$2,227,154"
9,DC,0,0,0,0,$0,$0,$0
10,DE,"15,619","45,997","6,044","52,041","$899,857","$499,867","$1,399,724"


In [399]:
df = pd.read_csv("us-states-territories.csv", encoding_errors="ignore")

df = df.rename(columns={
    "Abbreviation": "State"
})

df

# MERGE DATAFRAMES TOGETHER BASED ON COMMON COLUMN
# merged = df.merge(tables[0].df, left_on="State", right_on="Population (2019)")
# merged

,Type,Name,State,Capital,Population (2015),Population (2019),area (square miles)
0,State,Alabama,AL,Montgomery,NaN,"4,903,185","52,420"
1,State,Alaska,AK,Juneau,NaN,"731,545","665,384"
2,State,Arizona,AZ,Phoenix,NaN,"7,278,717","113,990"
3,State,Arkansas,AR,Little Rock,NaN,"3,017,804","53,179"
4,State,California,CA,Sacramento,NaN,"39,512,223","163,695"
...,...,...,...,...,...,...,...
60,Territory,Kingman Reef,NaN,NaN,NaN,NaN,0.005
61,Territory,Midway Atoll,NaN,NaN,NaN,NaN,3
62,Territory,Navassa Island,NaN,NaN,NaN,NaN,3
63,Territory,Palmyra Atoll,NaN,NaN,NaN,NaN,1.5


In [ ]:
# merged["Per capita"] = merged["Paid Hunting License Holders"] / merged["Population (2019)"]

# Not-Camelot questions

You can answer these without using Camelot.

## Federal rules on assault weapons

Download all of the PDFs from the Bureau of Alcohol, Tobacco, Firearms and Explosives's [Rules and Regulations Library](https://www.atf.gov/rules-and-regulations/rules-and-regulations-library). Filter for a list of all PDFs that contain the word `assault weapon` or `assault rifle`.

> If you're having trouble scraping, maybe someone will be kind enough to drop a list of PDF urls in Slack?

In [21]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [390]:
response = requests.get("https://www.atf.gov/rules-and-regulations/rules-and-regulations-library")
doc = BeautifulSoup(response.text)
doc

<!DOCTYPE html>
<!--[if IEMobile 7]><html class="no-js ie iem7 not-responsive" lang="en" dir="ltr"><![endif]--><!--[if lte IE 6]><html class="no-js ie lt-ie9 lt-ie8 lt-ie7 not-responsive" lang="en" dir="ltr"><![endif]--><!--[if (IE 7)&(!IEMobile)]><html class="no-js ie lt-ie9 lt-ie8 not-responsive" lang="en" dir="ltr"><![endif]--><!--[if IE 8]><html class="no-js ie lt-ie9 not-responsive" lang="en" dir="ltr"><![endif]--><!--[if (gte IE 9)|(gt IEMobile 7)]><html class="no-js ie responsive" lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema#"><![endif]--><!--[if !IE]><!--><html class="no-js responsive" dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://p

In [401]:
pdfs = []

table = doc.select("table tr", class_="views-table cols-4")
base_url = "https://www.atf.gov"

for row in table[1:]:
    if row.a != None:
        url = base_url + row.a.get("href")
        
        pdfs.append(url)
        
print(pdfs)

['https://www.atf.gov/file/168286/download', 'https://www.atf.gov/file/168281/download', 'https://www.atf.gov/file/55251/download', 'https://www.atf.gov/file/55546/download', 'https://www.atf.gov/file/55551/download', 'https://www.atf.gov/file/97816/download', 'https://www.atf.gov/file/97826/download', 'https://www.atf.gov/file/97831/download', 'https://www.atf.gov/file/97841/download', 'https://www.atf.gov/file/97856/download', 'https://www.atf.gov/file/97876/download', 'https://www.atf.gov/file/97881/download', 'https://www.atf.gov/file/97891/download', 'https://www.atf.gov/file/97896/download', 'https://www.atf.gov/file/97901/download', 'https://www.atf.gov/file/105771/download', 'https://www.atf.gov/file/124586/download', 'https://www.atf.gov/file/128221/download', 'https://www.atf.gov/file/165811/download', 'https://www.atf.gov/file/165581/download', 'https://www.atf.gov/file/165216/download', 'https://www.atf.gov/file/164886/download', 'https://www.atf.gov/file/164721/download', 

In [402]:
file_content = '\n'.join(pdfs)

with open("pdfs.txt", "w") as f:
    f.write(file_content)

In [404]:
!wget -i pdfs.txt

--2022-07-18 01:42:15--  https://www.atf.gov/file/168286/download
Resolving www.atf.gov (www.atf.gov)... 23.66.64.77
Connecting to www.atf.gov (www.atf.gov)|23.66.64.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815908 (797K) [application/pdf]
Saving to: ‘download’

download            100%[===================>] 796.79K  1.05MB/s    in 0.7s    

2022-07-18 01:42:16 (1.05 MB/s) - ‘download’ saved [815908/815908]

--2022-07-18 01:42:16--  https://www.atf.gov/file/168281/download
Reusing existing connection to www.atf.gov:443.
HTTP request sent, awaiting response... 200 OK
Length: 856992 (837K) [application/pdf]
Saving to: ‘download.1’

download.1          100%[===================>] 836.91K  1.06MB/s    in 0.8s    

2022-07-18 01:42:17 (1.06 MB/s) - ‘download.1’ saved [856992/856992]

--2022-07-18 01:42:17--  https://www.atf.gov/file/55251/download
Reusing existing connection to www.atf.gov:443.
HTTP request sent, awaiting response... 200 OK
Length: 68231 (6

In [ ]:
# for pdf in pdfs:
#     pdf.find("assault weapon")
# pdf.find("assault rifle")

# pdfs.find("assault weapon")
# # FILTER PDFS FOR ASSAULT WEAPON OR ASSAULT RIFLE

## New immigration judge training materials

Extract the text from [this 2020 guide for new immigration judges](2020-training-materials-2a-201-300.pdf) and save it as a file called `training-material.txt`.

> I took this PDF from [a FOIA request](https://www.muckrock.com/foi/united-states-of-america-10/most-recent-new-immigration-judge-training-materials-120125/#comms) – but the unfortunate thing is *I actually had to remove the OCR layer to make it part of this assignment*. By default everything that goes through Muckrock gets all of the text detected!

In [24]:
!pip install pytesseract

You should consider upgrading via the '/Users/onlyandrewn/.pyenv/versions/3.10.3/bin/python3.10 -m pip install --upgrade pip' command.


In [25]:
# !pip install Pillow
# !pip install pytesseract
# !pip install pdf2image

In [26]:
from PIL import Image
import pytesseract
from pdf2image import convert_from_path

In [27]:
# https://www.gcptutorials.com/post/python-extract-text-from-pdf-files
# https://towardsdatascience.com/extracting-text-from-scanned-pdf-using-pytesseract-open-cv-cd670ee38052

# poppler_path = ""
# pdf_path = "2020-training-materials-2a-201-300.pdf"

# images = convert_from_path(pdf_path=pdf_path, poppler_path=poppler_path)

# for count, img in enumerate(images):
#   img_name = f"page_{count}.png"  
#   img.save(img_name, "PNG")

# text = pytesseract.image_to_string('TKTK')
# print(text)

In [28]:
# with open(writePath, 'a') as f:
#     dfAsString = df.to_string(header=False, index=False)
#     f.write(dfAsString)

# df.to_string('training-material.txt', index = False)
# f = open("training-material.txt",'w')  # write in text mode